In [1]:
# Execute before using this notebook if using google colab

kernel = str(get_ipython())

if 'google.colab' in kernel:    
    !wget https://raw.githubusercontent.com/fredzett/rmqa/master/utils.py -P local_modules -nc 
    !npx degit fredzett/rmqa/data data
    import sys
    sys.path.append('local_modules')

In [2]:
import numpy as np
import statsmodels.formula.api as smf
import statsmodels.api as sm
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from utils import Datasets

**Important**

We will introduce and use two additional python modules from now on:

1. `Pandas`: handling and manipulating tabular data 

2. `Statsmodel`: statistical package for model building (e.g. regression models)

#  Linear regression

We will be using the advertising dataset from [Introduction to Statistical Learning](http://faculty.marshall.usc.edu/gareth-james/ISL/index.html) for the following chapter. 

Unlike in previous chapters we will load the dataset as a `pandas dataframe` instead of a numpy array. This will make data handling and visual inspection much easier. Also `pandas` works very well with other statistical package, e.g. statsmodel. 

The dataset contains 

- sales in thousands units
- advertising budgets in thousand of dollars for TV, radio and newspaper

with $200$ data points in total

In [3]:
df = pd.read_csv("data/Advertising.csv") # read csv file into pandas
df.head() # show first 5 items

,Unnamed: 0,TV,radio,newspaper,sales
0,1,230.1,37.8,69.2,22.1
1,2,44.5,39.3,45.1,10.4
2,3,17.2,45.9,69.3,9.3
3,4,151.5,41.3,58.5,18.5
4,5,180.8,10.8,58.4,12.9


In [4]:
df = df.drop(columns="Unnamed: 0") # we don't need the index column from the original file
df.head() # show first 5 items

,TV,radio,newspaper,sales
0,230.1,37.8,69.2,22.1
1,44.5,39.3,45.1,10.4
2,17.2,45.9,69.3,9.3
3,151.5,41.3,58.5,18.5
4,180.8,10.8,58.4,12.9


Linear regression is the standard topic of almost any statistical textbook. Although it is considered as a basic statistical method it is a very powerful tool in statistics and a very good starting point when conducting statistical analysis. For many more advanced method linear regression should serve as a benchmark. 

Purpose of this notebook is therefore to:

1. understand linear regression in depth

2. implement linear regression in python

Imagine we are the manager of the finance division and we want to understand the budget of the marketing unit. We may ask ourselves:

1. Is there a relation between advertising spent and sales?

2. Should we rather advertise via radio or via tv in order to increase sales?

3. Can we predict future sales with advertising data?

...

All these questions can be adressed with linear regression (amongst others). 

## Simple Linear Regression

Asssumes that there is a linear relationship between $X$ and $Y$ such that 

$$Y \approx \beta_0 + \beta_1X$$ 

or 

$$Y = \beta_0 + \beta_1 + \epsilon$$ 

where

$\beta_0$ and $\beta_1$ are the slope and the intercept in this linear model and are commonly referred to as the *coefficients* (or parameters) of the linear model and $\epsilon$ is an error term of form $N(0,1)$ and represents everything that is not captured by the specified population relationship. 


Given the above equation in practice is unknow, we will use our sample (advertising) data to calculate estimates of both coefficients such that: 

$$\hat{y} = \hat{\beta}_0 + \hat{\beta}_1x $$

where $\hat{y}$ denotes the prediction of $Y$ given $X = x$. 

$\hat{\beta}_0$ and $\hat{\beta}_1$ are the coefficient estimates. We will use our data (in which $Y$ is known) to estimate both coefficients such that $\hat{y}$ is close to $Y$ for each data point. 

Commonly, we do this by minimizing the *least squares* criterion, i.e. we want to minimize the resdiual sum of squares 

$$
\text{RSS} = \sum_{i=1}^n e_i^2
$$

where $e_i$ is defined as 

$$
e_i = y_i - \hat{y}_i
$$



This leads to the below $\beta_0$ and $beta_1$ that minimizes the RSS:

\begin{equation}
\begin{split}
\hat{\beta}_1 & = \frac{\sum_{i=1}^n (x_i - \bar{x})(y_i - \bar{y})}{\sum_{i=1}^n (x_i - \bar{x})^2} \\[10pt]
\hat{\beta}_0 & = \bar{y} - \hat{\beta}_1\bar{x}
\end{split}
\end{equation}


**Let's have an interactive look how this works: [click me](https://share.streamlit.io/fredzett/rmqa/regression.py)**

### using python

There are several different ways you can model linear relationships in python. Generally speaking we have to distinguish between three approaches:

1. Manual implementation using pure python

2. Supported implementation using statistical packages. Here we have to distinguish between:

   a. packages focusing on inference (more research / statistics focused)
    
   b. packages focusing on prediction (more machine learning focused)
 


Our focus will be on 1 (to test our understanding, practice python) and 2a (to make our lives easier). 

Let's assume we want to understand the relation between 

- tv advertisment spent (TV) and

- units of sales (sales)

We assume there is a linear relationship between both variables in the following form:

$$\text{sales} = \beta_0 + \beta_1 \text{TV}$$

In order to calculate the model we need to determine both coefficients. 

**1. Manual implementation:**

In [71]:
# Write functions for b0, b1 and ols

def _beta1(x,y):
    return np.sum((x - np.mean(x))*(y - np.mean(y))) / np.sum((x - np.mean(x))**2)

def _beta0(x,y):
    b1 = _beta1(x,y)
    return np.mean(y) - b1*np.mean(x)

def fit_ols(x,y):
    b1 = _beta1(x,y)
    b0 = _beta0(x,y)
    return b0, b1

In [72]:
x = df["TV"]
y = df["sales"]
fit_ols(x,y)

(7.032593549127698, 0.047536640433019736)

While this wasn't to hard, this approach has some disadvantages:

- yields only coefficients as outputs, but does not provide us with info of "how good" model is

- implementation with more features still missing

- manual implementation seems tedious and error prone

Luckikly we can use statistical packages to help us with all the constraints above. We will be using the module `statsmodels`

**2. Statsmodel implementation**

As with `scipy.stats` the module provides us with a nice and uniform approach to model all kinds of relationships (we will see this in later chapters). The steps we need to conduct are the following:

1. define a model + data

2. fit the model (i.e. optimize the coefficients)

The result is an object which can provide you with lots of functionality, e.g. 

- summary statistics

- prediction functionality for out of sample, i.e. new data

There are two approaches how to set up models:

1. formula api: where you type the model as a text

2. regular api: where you specify the model via passing the relevant dataframe

In [73]:
import statsmodels.formula.api as smf

In [128]:
ols = smf.ols("sales ~ TV", data=df) # Define model + data 
model = ols.fit() # fit the model

In [129]:
model.params # give coefficients

Intercept    7.032594
TV           0.047537
dtype: float64

In [130]:
new_data = [1, 23.23,104.25]
model.predict({"TV":new_data}) # predict sales for new data

0     7.080130
1     8.136870
2    11.988288
dtype: float64

Alternatively, you can use the standard api

In [131]:
import statsmodels.api as sm

In [134]:
x = df["TV"]
X = sm.add_constant(x) # add constant for intercept 
X.head()

,const,TV
0,1.0,230.1
1,1.0,44.5
2,1.0,17.2
3,1.0,151.5
4,1.0,180.8


ERROR! Session/line number was not unique in database. History logging moved to new session 535


In [135]:
ols = sm.OLS(y,X)
model = ols.fit()

In [138]:
model.params # give coefficients

const    7.032594
TV       0.047537
dtype: float64

In [137]:
new_data = [1, 23.23,104.25]
new_data_const = sm.add_constant(new_data)
model.predict(new_data_const) # predict sales for new data

array([ 7.08013019,  8.13686971, 11.98828831])

## Multiple linear regression

In practice we often find relationships that are more complex and thus have more than one predictors. 

Above we have analyzed the relationship between `sales` and `TV` advertisment. In reality the relationship may be more complex and could (most likely always will) also depend on other data. In our example we have also data for  `newspaper` and `radio` advertisment. Obviously, we could now go back and calculate individual single models on sales / newspaper or sales / radio. This has two disadvantages, however:

- how should the results from three individual analysis combined to one statement regarding the relationship between sales and advertisment?

- it may ignore effects from other advertisment efforts: when regressing sales on TV it ignores, e.g., the effect of tv advertisment on sales when also doing newspaper and/or radio advertisment. 

This is why we commonly include all variables into one model by extending the simple linear regression model to:

$$Y = \beta_0 + \beta_1X_1 + \beta_2X_2 + \ldots + \beta_nX_n + \epsilon$$

In our case our model would be described as:

$$sales = \beta_0 + \beta_1\text{TV} + \beta_2\text{newspaper} + \beta_3\text{radio}$$

Estimation of the $\beta$-coefficients follows the same approach as before, i.e. we minimize $RSS$, which is again given by:

$$
\text{RSS} = \sum_{i=1}^n e_i^2
$$

where $e_i$ is defined as 

$$
e_i = y_i - \hat{y}_i
$$

Only difference is that $\hat{y}$ is now calculated differently. 

Solving the optimization problem becomes somewhat more complicated than in the simple linear regression case as it involves solving a system of linear equations of the following form:

$$Y = X\beta + \epsilon$$

where:

\begin{equation}
Y = \begin{pmatrix} y_1 \\ y_2 \\ \vdots \\ y_n \end{pmatrix}, 
X = \begin{pmatrix} 1 & x_{11} & x_{12} & \ldots & x_{1p} \\ 1 & x_{21} & x_{22} & \ldots & x_{2p} \\ \vdots & \vdots &  \vdots  & \vdots & \vdots \\ 1 & x_{n1} & x_{n2} & \ldots & x_{np} \end{pmatrix},
\beta = \begin{pmatrix} \beta_0 \\ \vdots \\ \beta_p  \end{pmatrix}, 
\epsilon = \begin{pmatrix} \epsilon_0 \\ \epsilon_1  \\ \vdots \\ \epsilon_n  \end{pmatrix}, 
\end{equation}

Without covering the details of the math this equation can be solved as follows:

$$
\hat{\beta} = (X^TX)^{-1}X^Ty
$$

What is important here is that there exists an analytical solution for the linear regression problem. 

We can easily implement this solution using `numpy`

### using python

In [99]:
def fit_ols_multiple(X,y):
    return np.linalg.inv(X.T@X)@X.T@y

In [139]:
X = df[["TV","newspaper","radio"]].values # convert to numpy
X = sm.add_constant(X)
y = df["sales"].values # convert to numpy

fit_ols_multiple(X,y)

array([ 2.93888937e+00,  4.57646455e-02, -1.03749304e-03,  1.88530017e-01])

In [124]:
ols = smf.ols("sales ~ TV + newspaper + radio",df)
model = ols.fit()
model.params # or model.summary for all other info

Intercept    2.938889
TV           0.045765
newspaper   -0.001037
radio        0.188530
dtype: float64

## Assessing results & checking assumptions

So far we have covered how to **estimate** the regression coefficient for simple and multiple regression analysis. 

However, in particular when doing research it is important to do the following three things in addition:

- check coefficient accuracy

- check model accuracy

- check if assumptions of model hold, i.e. if model specification is correct for the problem at hand

We will continue to use the advertisment data

In [205]:
df.head()

,TV,radio,newspaper,sales
0,230.1,37.8,69.2,22.1
1,44.5,39.3,45.1,10.4
2,17.2,45.9,69.3,9.3
3,151.5,41.3,58.5,18.5
4,180.8,10.8,58.4,12.9


In [142]:
model = smf.ols("sales ~ TV + radio + newspaper", df).fit()

### Check coefficient accuracy

In [145]:
betas = model.params
betas

Intercept    2.938889
TV           0.045765
radio        0.188530
newspaper   -0.001037
dtype: float64

In [245]:
def bootstrap(df, f= "sales ~TV + radio + newspaper", coefs=4, sims=1000):
    n = len(df)
    sims, p = sims, coefs # number of simulation, number of coefficients
    all_betas = np.empty((sims,p))
    for i in range(sims):
        smpl = df.sample(n, replace=True)
        betas = smf.ols(f, smpl).fit().params
        all_betas[i, :] = betas
    return all_betas

In [ ]:
betas = bootstrap(df)

In [250]:
np.quantile(betas[:,1], q=[0.025, 0.975])

array([0.04195758, 0.04962702])

In [171]:
smf.ols("sales ~ TV + radio + newspaper", df).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  sales   R-squared:                       0.897
Model:                            OLS   Adj. R-squared:                  0.896
Method:                 Least Squares   F-statistic:                     570.3
Date:                Fri, 30 Oct 2020   Prob (F-statistic):           1.58e-96
Time:                        14:00:46   Log-Likelihood:                -386.18
No. Observations:                 200   AIC:                             780.4
Df Residuals:                     196   BIC:                             793.6
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      2.9389      0.312      9.422      0.000       2.324       3.554
TV             0.0458      0.001     32.809      0.000       0.043       0.049
radio          0.1885      0.009     21.893      0.000       0.172       0.206
newspaper     -0.0010      0.006     -0.177      0.860      -0.013       0.011
==============================================================================
Omnibus:                       60.414   Durbin-Watson:                   2.084
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              151.241
Skew:                          -1.327   Prob(JB):                     1.44e-33
Kurtosis:                       6.332   Cond. No.                         454.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [244]:
X = df[["TV","radio","newspaper"]]
X = sm.add_constant(X)
np.sqrt(np.diag(np.linalg.inv(X.T@X)))

array([0.18505269, 0.00082758, 0.00510898, 0.00348322])

In [243]:
model = smf.ols("sales ~ TV + radio + newspaper", df).fit()
betas = model.params
e = model.resid

np.sqrt(np.diag(model.cov_params()))

array([0.31190824, 0.0013949 , 0.00861123, 0.00587101])